Grupo 12

Francisco de Borja Lozano del Moral

Manuel Ortega Salvador

# Código para el problema 4 - Grados de separación entre actores
## Definición del problema
Explicamos la elección de la representación de estados:
Únicamente necesitamos conocer el id de la persona, ya que a raíz de ese id podemos obtener el id de las películas en las que participó, y con ello todos los actores con los que ha trabajado (neighbors_for_person), obteniendo así el grafo de exploración. De esta manera los estados son los id de los actores y las acciones son, de nuevo, los vecinos de los actores.

De esta manera, el problema se vuelve la búsqueda del camino más corto entre dos nodos en un grafo, donde cada arista tiene el mismo peso. Por esto, el algoritmo que más cuadra para este problema es Dijkstra's (breadth_first_graph_search). Va a haber ciclos por todo el grafo, luego cabe esperar que breadth_first_tree_search sea más lento y además, si no hay solución, se quede atascado.

No hay ninguna razón para creer que depth_first_graph_search vaya a funcionar bien, ya que no solo queremos llegar al nodo buscado, también lograr el camino más corto, lo cual no tiene por que ser cierto ni para DFS ni para A* con heurísticas que el primer nodo camino al objetivo encontrado sea el óptimo. 

Con A* algunas heurísticas aseguran que se preserva la optimalidad de la primera solución encontrada, pero en este caso, no hay manera de estimar la distancia entre dos nodos del grafo de alguna manera que no requiera la exploración del grafo.

Si lo que se quisiera fuera encontrar *cualquier* solución, entonces tendría sentido probar DFS o A*, con una heurística como la cantidad de películas hechas por el nodo, ya que los actores más activos son los que más probablemente han colaborado con otros actores.

Pero esto no es lo que se ha pedido en este ejercicio, luego no vamos a explorar estas opciones.

### Ánalisis de la complejidad
Resolveremos el problema empleando la búsqueda en anchura, puesto que queremos encontrar el camino más corto. Como sabemos, tiene una complejidad O(r^p), siendo r el número de hijos para un nodo y p la profundidad de la solución.

El caso mejor sería encontrar la solución a profundidad 1, luego sería lineal respecto al número de hijos de la raíz.

In [1]:
## grados.py
import csv
import sys

# diccionario de nombres de personas con ids 
names = {}
# diccionario: name, birth, movies (conjunto de movie_ids)
people = {}
# movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Cargamos el archivo people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # cargamos el archivo movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # cargamos el archivo stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def shortest_path(source, target):
    """
    Devuelve la lista de pares (movie_id, person_id) que conectan source y target o None si hay conexion.
    """
    p = ProblemaGrado(source, target)
    path = breadth_first_tree_search(p).solution()
    
    if len(path) == 0:
        return None
    else:
        return path

def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors

In [40]:
#cargamos los datos
load_data("small")

In [41]:
load_data("large") 

In [42]:
## Tests
# name="Emma Watson"
# person_id=person_id_for_name(name)
# print(person_id_for_name("Daniel Radcliffe"))
# neighbors_for_person(person_id)

## Definimos la clase Problema

In [45]:
from search import *

In [46]:
class ProblemaGrado(Problem):
    def _init_(self, initial, goal):
        Problem._init(self,initial,goal)
    
    def actions(self, state):
        return neighbors_for_person(state)
    
    def result(self, state, action):
        return action[1]
    

In [47]:
class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
def resuelve_grado(source, target, algoritmo, h=None):
    p_analizado = Problema_con_Analizados(ProblemaGrado(source, target))
    if h: 
        sol= algoritmo(p_analizado,h).solution()
    else: 
        sol= algoritmo(p_analizado).solution()
    print("Solución: {0}".format(sol))
    print("Algoritmo: {0}".format(algoritmo.__name__))
    if h: 
        print("Heurística: {0}".format(h.__name__))
    else:
        pass
    print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p_analizado.analizados))

In [48]:
source = person_id_for_name("Emma Watson")
target = person_id_for_name("Jennifer Lawrence")

In [63]:
resuelve_grado(source1,target1,breadth_first_graph_search)

Solución: [('373889', '705356'), ('1976009', '564215'), ('6565702', '2225369')]
Algoritmo: breadth_first_graph_search
Longitud de la solución: 3. Nodos analizados: 2226


## Para ejecutar el main

In [ ]:
source = person_id_for_name(input("Nombre: "))
if source is None:
    sys.exit("Esa persona no se encuentra.")
target = person_id_for_name(input("Nombre: "))
if target is None:
    sys.exit("Esa persona no se encuentra.")

path = shortest_path(source, target)

if path is None:
    print("No están conectados.")
else:
    degrees = len(path)
    print(f"{degrees} grados de separacion.")
    path = [(None, source)] + path
    for i in range(degrees):
        person1 = people[path[i][1]]["name"]
        person2 = people[path[i + 1][1]]["name"]
        movie = movies[path[i + 1][0]]["title"]
        print(f"{i + 1}: {person1} y {person2} participaron en {movie}")

Nombre: Kevin Bacon
Which 'Kevin Bacon'?
ID: 9323132, Name: Kevin Bacon, Birth: 
ID: 102, Name: Kevin Bacon, Birth: 1958
Intended Person ID: 102
Nombre: Tommy Wiseau
